## Experimento 2.
## Predicción de permanencia de migrantes mexicanos en situación irregular en Estados Unidos tras la promulgación de la $\textit{IIRIRA}$


Selección y limpieza de datos a partir de las tablas <b>MIG</b> y <b>MIGOTHER</b>.

La tabla <b>MIG</b> contiene información a nivel personal y detalles sobre todos los cruces indocumentados (hasta 30) de cada jefe de familia, así como toda la actividad económica y social durante el último viaje migratorio a los Estados Unidos.

La tabla <b>MIGOTHER</b> contiene la misma información pero considera otra persona del hogar que emigró a los Estados Unidos en aquellos hogares donde el jefe del hogar no tuvo experiencia migratoria.

El dataset se forma a partir de la combinación de ambas tablas considerando aquellas columnas que comparten. (Ver documentación para detalles)

Para este experimento se considera solamente información pertinente al último viaje migratorio realizado, tomando como etiqueta de clase al campo 'uscurtrp' que responde a la pregunta ¿Actualmente se está en último viaje migratorio?.

Así, la etiqueta de clase puede tomar dos valores posibles:<br>
>1: Sí (No ha regresado)<br>2: No (Ya regresó)<br>

Para todos los clasificadores utilizados, se tomará la clase 1 (No ha regresado) como clase **Positive**.

<br>

---

Bibliotecas utilizadas

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cpi #Paquete para ajustar salarios y precios en USD de acuerdo a los índices de inflación del último año que se realizó la encuesta	
import os
import time
from multiprocessing import Pool, cpu_count

Rutas de acceso y almacenamiento de los archivos generados

In [ ]:
#Ruta para leer el archivo a procesar
path_to_data = os.path.join('../'*4, 'Bases de datos', 'Princeton', 'Mexican Migration Project (MMP)', 'CSV', 'core_files')

#Ruta para almacenar el dataset procesado
path_to_store = '~/Documents/Tesis/Experimentos/MMP/datasets/regreso_ultimo_viaje'

Tipos de dato para cada atributo en las tablas

In [3]:
dtypes_mig = {'country': 'category', 'commun': 'category', 'surveypl': 'category', 'surveyyr': 'category', 'hhnum': 'category', 'sex': 'category', 'yrborn': 'category', 'placebrn': 'category', 
              'statebrn': 'category', 'marstat': 'category', 'edyrs': 'category', 'occ': 'category', '@by': 'category', 'smokes': 'category', 'smoked': 'category', 'smokeage': 'category',
              'health': 'category', 'healthly': 'category', 'healthnw': 'category', 'hyperten': 'category', 'sugar': 'category', 'heart': 'category', 'brain': 'category', 'lung': 'category', 
              'nerves': 'category', 'cancer': 'category', 'hypertyr': 'category', 'sugaryr': 'category', 'heartyr': 'category', 'brainyr': 'category', 'lungyr': 'category', 'nervesyr': 'category', 
              'canceryr': 'category', 'usyr1': 'category', 'usmar1': 'category', 'usdoc1': 'category', 'usplace1': 'category', 'usstate1': 'category', 'usocc1': 'category', 'usby1': 'category', 
              'usyrl': 'category', 'usmarl': 'category', 'usdocl': 'category', 'usplacel': 'category', 'usstatel': 'category', 'usoccl': 'category', 'usbyl': 'category', 'uscurtrp': 'category', 
              'ushealth': 'category', 'mxhealth': 'category', 'legyrapp': 'category', 'legyrrec': 'category', 'legspon': 'category', 'cityrapp': 'category', 'cityrrec': 'category', 'dacaapp': 'category',
              'dacarec': 'category', 'crsyr1': 'category', 'crspl1': 'category', 'crsst1': 'category', 'crshow1': 'category', 'crscoy1': 'category', 'crspay1': 'category', 'crsdep1': 'category', 
              'crsyes1': 'category', 'crsyr2': 'category', 'crspl2': 'category', 'crsst2': 'category', 'crshow2': 'category', 'crscoy2': 'category', 'crspay2': 'category', 'crsdep2': 'category',
              'crsyes2': 'category', 'crsyr3': 'category', 'crspl3': 'category', 'crsst3': 'category', 'crshow3': 'category', 'crscoy3': 'category', 'crspay3': 'category', 'crsdep3': 'category', 
              'crsyes3': 'category', 'crsyr4': 'category', 'crspl4': 'category', 'crsst4': 'category', 'crshow4': 'category', 'crscoy4': 'category', 'crspay4': 'category', 'crsdep4': 'category', 
              'crsyes4': 'category', 'crsyr5': 'category', 'crspl5': 'category', 'crsst5': 'category', 'crshow5': 'category', 'crscoy5': 'category', 'crspay5': 'category', 'crsdep5': 'category', 
              'crsyes5': 'category', 'crsyr6': 'category', 'crspl6': 'category', 'crsst6': 'category', 'crshow6': 'category', 'crscoy6': 'category', 'crspay6': 'category', 'crsdep6': 'category', 
              'crsyes6': 'category', 'crsyr7': 'category', 'crspl7': 'category', 'crsst7': 'category', 'crshow7': 'category', 'crscoy7': 'category', 'crspay7': 'category', 'crsdep7': 'category',
              'crsyes7': 'category', 'crsyr8': 'category', 'crspl8': 'category', 'crsst8': 'category', 'crshow8': 'category', 'crscoy8': 'category', 'crspay8': 'category', 'crsdep8': 'category',
              'crsyes8': 'category', 'crsyr9': 'category', 'crspl9': 'category', 'crsst9': 'category', 'crshow9': 'category', 'crscoy9': 'category', 'crspay9': 'category', 'crsdep9': 'category', 
              'crsyes9': 'category', 'crsyr10': 'category', 'crspl10': 'category', 'crsst10': 'category', 'crshow10': 'category', 'crscoy10': 'category', 'crspay10': 'category', 
              'crsdep10': 'category', 'crsyes10': 'category', 'crsyr11': 'category', 'crspl11': 'category', 'crsst11': 'category', 'crshow11': 'category', 'crscoy11': 'category', 
              'crspay11': 'category', 'crsdep11': 'category', 'crsyes11': 'category', 'crsyr12': 'category', 'crspl12': 'category', 'crsst12': 'category', 'crshow12': 'category', 
              'crscoy12': 'category', 'crspay12': 'category', 'crsdep12': 'category', 'crsyes12': 'category', 'crsyr13': 'category', 'crspl13': 'category', 'crsst13': 'category', 
              'crshow13': 'category', 'crscoy13': 'category', 'crspay13': 'category', 'crsdep13': 'category', 'crsyes13': 'category', 'crsyr14': 'category', 'crspl14': 'category', 
              'crsst14': 'category', 'crshow14': 'category', 'crscoy14': 'category', 'crspay14': 'category', 'crsdep14': 'category', 'crsyes14': 'category', 'crsyr15': 'category', 
              'crspl15': 'category', 'crsst15': 'category', 'crshow15': 'category', 'crscoy15': 'category', 'crspay15': 'category', 'crsdep15': 'category', 'crsyes15': 'category',
              'crsyr16': 'category', 'crspl16': 'category', 'crsst16': 'category', 'crshow16': 'category', 'crscoy16': 'category', 'crspay16': 'category', 'crsdep16': 'category', 
              'crsyes16': 'category', 'crsyr17': 'category', 'crspl17': 'category', 'crsst17': 'category', 'crshow17': 'category', 'crscoy17': 'category', 'crspay17': 'category',
              'crsdep17': 'category', 'crsyes17': 'category', 'crsyr18': 'category', 'crspl18': 'category', 'crsst18': 'category', 'crshow18': 'category', 'crscoy18': 'category',
              'crspay18': 'category', 'crsdep18': 'category', 'crsyes18': 'category', 'crsyr19': 'category', 'crspl19': 'category', 'crsst19': 'category', 'crshow19': 'category',
              'crscoy19': 'category', 'crspay19': 'category', 'crsdep19': 'category', 'crsyes19': 'category', 'crsyr20': 'category', 'crspl20': 'category', 'crsst20': 'category',
              'crshow20': 'category', 'crscoy20': 'category', 'crspay20': 'category', 'crsdep20': 'category', 'crsyes20': 'category', 'crsyr21': 'category', 'crspl21': 'category',
              'crsst21': 'category', 'crshow21': 'category', 'crscoy21': 'category', 'crspay21': 'category', 'crsdep21': 'category', 'crsyes21': 'category', 'crsyr22': 'category',
              'crspl22': 'category', 'crsst22': 'category', 'crshow22': 'category', 'crscoy22': 'category', 'crspay22': 'category', 'crsdep22': 'category', 'crsyes22': 'category',
              'crsyr23': 'category', 'crspl23': 'category', 'crsst23': 'category', 'crshow23': 'category', 'crscoy23': 'category', 'crspay23': 'category', 'crsdep23': 'category',
              'crsyes23': 'category', 'crsyr24': 'category', 'crspl24': 'category', 'crsst24': 'category', 'crshow24': 'category', 'crscoy24': 'category', 'crspay24': 'category',
              'crsdep24': 'category', 'crsyes24': 'category', 'crsyr25': 'category', 'crspl25': 'category', 'crsst25': 'category', 'crshow25': 'category', 'crscoy25': 'category',
              'crspay25': 'category', 'crsdep25': 'category', 'crsyes25': 'category', 'crsyr26': 'category', 'crspl26': 'category', 'crsst26': 'category', 'crshow26': 'category',
              'crscoy26': 'category', 'crspay26': 'category', 'crsdep26': 'category', 'crsyes26': 'category', 'crsyr27': 'category', 'crspl27': 'category', 'crsst27': 'category',
              'crshow27': 'category', 'crscoy27': 'category', 'crspay27': 'category', 'crsdep27': 'category', 'crsyes27': 'category', 'crsyr28': 'category', 'crspl28': 'category',
              'crsst28': 'category', 'crshow28': 'category', 'crscoy28': 'category', 'crspay28': 'category', 'crsdep28': 'category', 'crsyes28': 'category', 'crsyr29': 'category',
              'crspl29': 'category', 'crsst29': 'category', 'crshow29': 'category', 'crscoy29': 'category', 'crspay29': 'category', 'crsdep29': 'category', 'crsyes29': 'category',
              'crsyr30': 'category', 'crspl30': 'category', 'crsst30': 'category', 'crshow30': 'category', 'crscoy30': 'category', 'crspay30': 'category', 'crsdep30': 'category',
              'crsyes30': 'category', 'momgyr': 'category', 'mogree': 'category', 'moyrgr': 'category', 'modoc': 'category', 'modoyr': 'category', 'molive': 'category', 'mousnw':
              'category', 'mouspl': 'category', 'mousst': 'category', 'famgyr': 'category', 'fagree': 'category', 'fayrgr': 'category', 'fadoc': 'category', 'fadoyr': 'category',
              'falive': 'category', 'fausnw': 'category', 'fauspl': 'category', 'fausst': 'category', 'sbsex1': 'category', 'sbmgyr1': 'category', 'sbgree1': 'category',
              'sbyrgr1': 'category', 'sbdoc1': 'category', 'sbdoyr1': 'category', 'sblive1': 'category', 'sbusnw1': 'category', 'sbuspl1': 'category', 'sbusst1': 'category', 
              'sbsex2': 'category', 'sbmgyr2': 'category', 'sbgree2': 'category', 'sbyrgr2': 'category', 'sbdoc2': 'category', 'sbdoyr2': 'category', 'sblive2': 'category', 'sbusnw2': 'category',
              'sbuspl2': 'category', 'sbusst2': 'category', 'sbsex3': 'category', 'sbmgyr3': 'category', 'sbgree3': 'category', 'sbyrgr3': 'category', 'sbdoc3': 'category', 'sbdoyr3': 'category',
              'sblive3': 'category', 'sbusnw3': 'category', 'sbuspl3': 'category', 'sbusst3': 'category', 'sbsex4': 'category', 'sbmgyr4': 'category', 'sbgree4': 'category', 'sbyrgr4': 'category',
              'sbdoc4': 'category', 'sbdoyr4': 'category', 'sblive4': 'category', 'sbusnw4': 'category', 'sbuspl4': 'category', 'sbusst4': 'category', 'sbsex5': 'category', 'sbmgyr5': 'category',
              'sbgree5': 'category', 'sbyrgr5': 'category', 'sbdoc5': 'category', 'sbdoyr5': 'category', 'sblive5': 'category', 'sbusnw5': 'category', 'sbuspl5': 'category', 'sbusst5': 'category',
              'sbsex6': 'category', 'sbmgyr6': 'category', 'sbgree6': 'category', 'sbyrgr6': 'category', 'sbdoc6': 'category', 'sbdoyr6': 'category', 'sblive6': 'category', 'sbusnw6': 'category',
              'sbuspl6': 'category', 'sbusst6': 'category', 'sbsex7': 'category', 'sbmgyr7': 'category', 'sbgree7': 'category', 'sbyrgr7': 'category', 'sbdoc7': 'category', 'sbdoyr7': 'category',
              'sblive7': 'category', 'sbusnw7': 'category', 'sbuspl7': 'category', 'sbusst7': 'category', 'sbsex8': 'category', 'sbmgyr8': 'category', 'sbgree8': 'category', 'sbyrgr8': 'category',
              'sbdoc8': 'category', 'sbdoyr8': 'category', 'sblive8': 'category', 'sbusnw8': 'category', 'sbuspl8': 'category', 'sbusst8': 'category', 'sbsex9': 'category', 'sbmgyr9': 'category',
              'sbgree9': 'category', 'sbyrgr9': 'category', 'sbdoc9': 'category', 'sbdoyr9': 'category', 'sblive9': 'category', 'sbusnw9': 'category', 'sbuspl9': 'category', 'sbusst9': 'category',
              'sbsex10': 'category', 'sbmgyr10': 'category', 'sbgree10': 'category', 'sbyrgr10': 'category', 'sbdoc10': 'category', 'sbdoyr10': 'category', 'sblive10': 'category', 'sbusnw10': 'category',
              'sbuspl10': 'category', 'sbusst10': 'category', 'sbsex11': 'category', 'sbmgyr11': 'category', 'sbgree11': 'category', 'sbyrgr11': 'category', 'sbdoc11': 'category', 'sbdoyr11': 'category',
              'sblive11': 'category', 'sbusnw11': 'category', 'sbuspl11': 'category', 'sbusst11': 'category', 'sbsex12': 'category', 'sbmgyr12': 'category', 'sbgree12': 'category', 'sbyrgr12': 'category',
              'sbdoc12': 'category', 'sbdoyr12': 'category', 'sblive12': 'category', 'sbusnw12': 'category', 'sbuspl12': 'category', 'sbusst12': 'category', 'uncinus': 'category', 'uncevus': 'category',
              'coinus': 'category', 'coevus': 'category', 'neinus': 'category', 'neevus': 'category', 'sbilinus': 'category', 'sbilevus': 'category', 'chilinus': 'category', 'chilevus': 'category',
              'pilinus': 'category','pilevus': 'category', 'frinus': 'category', 'frevus': 'category', 'reltrip1': 'category', 'reltrip': 'category', 'paistp1': 'category', 'paistrip': 'category',
              'sport': 'category', 'social': 'category', 'chicanos': 'category', 'blacks': 'category', 'asians': 'category', 'anglos': 'category', 'latinos': 'category', 'lodging1': 'category',
              'lodging': 'category', 'english': 'category', 'enghome': 'category', 'engwork': 'category', 'engfrnd': 'category', 'engneig': 'category', 'howjob': 'category', 'hrweek': 'category',
              'monsyr': 'category', 'howpaid': 'category', 'sstax': 'category', 'fedtx': 'category', 'rowner': 'category', 'rsuper': 'category', 'howjobs': 'category', 'hrweeks': 'category',
              'monsyrs': 'category', 'howpaids': 'category', 'sstaxs': 'category', 'fedtxs': 'category', 'rowners': 'category', 'rsupers': 'category', 'finhelp1': 'category',
              'finhelp': 'category', 'bankacct': 'category', 'card': 'category', 'taxes': 'category', 'consular': 'category', 'remit1': 'category', 'remit2': 'category', 'remit3': 'category',
              'remit4': 'category', 'remit5': 'category', 'savings1': 'category', 'savings2': 'category', 'savings3': 'category', 'savings4': 'category', 'savings5': 'category', 'schools': 'category',
              'unemp': 'category', 'foodst': 'category', 'welfare': 'category', 'afdc': 'category', 'wic': 'category', 'ssi': 'category', 'general': 'category', 'doctor': 'category', 'hospital': 'category',
              'hlthpmt1': 'category', 'hlthpmt2': 'category', 'hlthpmt3': 'category', 'hlthpmt4': 'category', 'apppaid': 'category', 'appcurr': 'category', 'visacurr': 'category', 'visawho': 'category',
              'doctaken': 'category', 'trippaid': 'category', 'tripcost': 'category', 'tripcurr': 'category', 'retpaid': 'category', 'retcurr': 'category', 'contract': 'category', 'conkept': 'category',
              'convio1': 'category', 'convio2': 'category', 'convio3': 'category', 'convio4': 'category', 'convio5': 'category', 'transwk': 'category', 'housing': 'category', 'houcurr': 'category',
              'houfreq': 'category', 'houshare': 'category', 'houpers': 'category', 'houtype': 'category', 'hkitchen': 'category', 'hdining': 'category', 'hliving': 'category', 'hbath': 'category',
              'hrooms': 'category', 'hcond': 'category', 'abused': 'category', 'abusewho': 'category', 'abuserep': 'category', 'accident': 'category', 'medical': 'category', 'medicpd': 'category',
              'localwkr': 'category', 'latinwkr': 'category', 'samejob': 'category', 'samepay': 'category', 'overtime': 'category', 'overtmtp': 'category', 'tempcomp': 'category', 'unempben': 'category',
              'embassy': 'category', 'unions': 'category', 'weight': 'float64', 'age': 'float64', 'hhincome': 'float64', 'ldowage': 'float64', 'height': 'float64', 'kgs': 'float64', 'usdur1': 'float64',
              'usdurl': 'float64', 'uswage1': 'float64', 'uswagel': 'float64', 'ustrips': 'float64', 'usexp': 'float64', 'crscst1': 'float64', 'crscst2': 'float64', 'crscst3': 'float64', 'crscst4': 'float64',
              'crscst5': 'float64', 'crscst6': 'float64', 'crscst7': 'float64', 'crscst8': 'float64', 'crscst9': 'float64', 'crscst10': 'float64', 'crscst11': 'float64', 'crscst12': 'float64',
              'crscst13': 'float64', 'crscst14': 'float64', 'crscst15': 'float64', 'crscst16': 'float64', 'crscst17': 'float64', 'crscst18': 'float64', 'crscst19': 'float64', 'crscst20': 'float64',
              'crscst21': 'float64', 'crscst22': 'float64', 'crscst23': 'float64', 'crscst24': 'float64', 'crscst25': 'float64', 'crscst26': 'float64', 'crscst27': 'float64', 'crscst28': 'float64',
              'crscst29': 'float64', 'crscst30': 'float64', 'hrwage': 'float64', 'hrwages': 'float64', 'rent': 'float64', 'food': 'float64', 'remit': 'float64', 'savings': 'float64', 'savretrn': 'float64',
              'appcost': 'float64', 'visacost': 'float64', 'retcost': 'float64', 'houcost': 'float64'}

dtypes_migother = {'country': 'category', 'commun': 'category', 'surveypl': 'category', 'surveyyr': 'category', 'hhnum': 'category', 'persnum': 'category', 'sex': 'category', 'yrborn': 'category', 
                   'placebrn': 'category', 'statebrn': 'category', 'marstat': 'category', 'edyrs': 'category', 'occ': 'category', 'usyr1': 'category', 'usdur1': 'category', 'usdoc1': 'category',
                   'usstate1': 'category', 'usplace1': 'category', 'usmar1': 'category', 'usocc1': 'category', 'usby1': 'category', 'usyrl': 'category', 'usdurl': 'category', 'usdocl': 'category',
                   'usstatel': 'category', 'usplacel': 'category', 'usmarl': 'category', 'usoccl': 'category', 'usbyl': 'category', 'uscurtrp': 'category', 'legyrapp': 'category', 'legyrrec': 'category',
                   'legspon': 'category', 'cityrapp': 'category', 'cityrrec': 'category', 'dacaapp': 'category', 'dacarec': 'category', 'crsyr1': 'category', 'crspl1': 'category', 'crsst1': 'category',
                   'crshow1': 'category', 'crscoy1': 'category', 'crspay1': 'category', 'crsdep1': 'category', 'crsyes1': 'category', 'crsyr2': 'category', 'crspl2': 'category', 'crsst2': 'category',
                   'crshow2': 'category', 'crscoy2': 'category', 'crspay2': 'category', 'crsdep2': 'category', 'crsyes2': 'category', 'crsyr3': 'category', 'crspl3': 'category', 'crsst3': 'category',
                   'crshow3': 'category', 'crscoy3': 'category', 'crspay3': 'category', 'crsdep3': 'category', 'crsyes3': 'category', 'crsyr4': 'category', 'crspl4': 'category', 'crsst4': 'category',
                   'crshow4': 'category', 'crscoy4': 'category', 'crspay4': 'category', 'crsdep4': 'category', 'crsyes4': 'category', 'crsyr5': 'category', 'crspl5': 'category', 'crsst5': 'category',
                   'crshow5': 'category', 'crscoy5': 'category', 'crspay5': 'category', 'crsdep5': 'category', 'crsyes5': 'category', 'crsyr6': 'category', 'crspl6': 'category', 'crsst6': 'category',
                   'crshow6': 'category', 'crscoy6': 'category', 'crspay6': 'category', 'crsdep6': 'category', 'crsyes6': 'category', 'crsyr7': 'category', 'crspl7': 'category', 'crsst7': 'category',
                   'crshow7': 'category', 'crscoy7': 'category', 'crspay7': 'category', 'crsdep7': 'category', 'crsyes7': 'category', 'crsyr8': 'category', 'crspl8': 'category', 'crsst8': 'category',
                   'crshow8': 'category', 'crscoy8': 'category', 'crspay8': 'category', 'crsdep8': 'category', 'crsyes8': 'category', 'crsyr9': 'category', 'crspl9': 'category', 'crsst9': 'category',
                   'crshow9': 'category', 'crscoy9': 'category', 'crspay9': 'category', 'crsdep9': 'category', 'crsyes9': 'category', 'crsyr10': 'category', 'crspl10': 'category', 'crsst10': 'category',
                   'crshow10': 'category', 'crscoy10': 'category', 'crspay10': 'category', 'crsdep10': 'category', 'crsyes10': 'category', 'reltrip1': 'category', 'reltrip': 'category',
                   'paistp1': 'category', 'paistrip': 'category', 'sport': 'category', 'social': 'category', 'chicanos': 'category', 'blacks': 'category', 'asians': 'category', 'anglos': 'category',
                   'latinos': 'category', 'lodging1': 'category', 'lodging': 'category', 'english': 'category', 'enghome': 'category', 'engwork': 'category', 'engfrnd': 'category', 'engneig': 'category',
                   'howjobo': 'category', 'hrweeko': 'category', 'monsyro': 'category', 'howpaido': 'category', 'sstaxo': 'category', 'fedtxo': 'category', 'rownero': 'category', 'rsupero': 'category',
                   'finhelp1': 'category', 'finhelp': 'category', 'bankacct': 'category', 'card': 'category', 'taxes': 'category', 'consular': 'category', 'remit1': 'category', 'remit2': 'category',
                   'remit3': 'category', 'remit4': 'category', 'remit5': 'category', 'savings1': 'category', 'savings2': 'category', 'savings3': 'category', 'savings4': 'category', 'savings5': 'category',
                   'schools': 'category', 'unemp': 'category', 'foodst': 'category', 'welfare': 'category', 'afdc': 'category', 'wic': 'category', 'ssi': 'category', 'general': 'category',
                   'doctor': 'category', 'hospital': 'category', 'hlthpmt1': 'category', 'hlthpmt2': 'category', 'hlthpmt3': 'category', 'hlthpmt4': 'category', 'apppaid': 'category', 'appcurr': 'category',
                   'visacurr': 'category', 'visawho': 'category', 'doctaken': 'category', 'trippaid': 'category', 'tripcurr': 'category', 'retpaid': 'category', 'retcurr': 'category', 'contract': 'category',
                   'conkept': 'category', 'convio1': 'category', 'convio2': 'category', 'convio3': 'category', 'convio4': 'category', 'convio5': 'category', 'transwk': 'category', 'housing': 'category',
                   'houcurr': 'category', 'houfreq': 'category', 'houshare': 'category', 'houpers': 'category', 'houtype': 'category', 'hkitchen': 'category', 'hdining': 'category', 'hliving': 'category',
                   'hbath': 'category', 'hrooms': 'category', 'hcond': 'category', 'abused': 'category', 'abusewho': 'category', 'abuserep': 'category', 'accident': 'category', 'medical': 'category',
                   'medicpd': 'category', 'localwkr': 'category', 'latinwkr': 'category', 'samejob': 'category', 'samepay': 'category', 'overtime': 'category', 'overtmtp': 'category', 'tempcomp': 'category',
                   'unempben': 'category', 'embassy': 'category', 'unions': 'category', 'weight': 'float64', 'age': 'float64', 'uswage1': 'float64', 'uswagel': 'float64', 'ustrips': 'float64',
                   'crscst1': 'float64', 'crscst2': 'float64', 'crscst3': 'float64', 'crscst4': 'float64', 'crscst5': 'float64', 'crscst6': 'float64', 'crscst7': 'float64', 'crscst8': 'float64',
                   'crscst9': 'float64', 'crscst10': 'float64', 'crscst11': 'float64', 'crscst12': 'float64', 'crscst13': 'float64', 'crscst14': 'float64', 'crscst15': 'float64', 'crscst16': 'float64',
                   'crscst17': 'float64', 'crscst18': 'float64', 'crscst19': 'float64', 'crscst20': 'float64', 'crscst21': 'float64', 'crscst22': 'float64', 'crscst23': 'float64', 'crscst24': 'float64',
                   'crscst25': 'float64', 'crscst26': 'float64', 'crscst27': 'float64', 'crscst28': 'float64', 'crscst29': 'float64', 'crscst30': 'float64', 'hrwageo': 'float64', 'rent': 'float64',
                   'food': 'float64', 'remit': 'float64', 'savings': 'float64', 'savretrn': 'float64', 'appcost': 'float64', 'visacost': 'float64', 'tripcost': 'float64', 'retcost': 'float64',
                   'houcost':'float64'}


In [4]:
#Selección de atributos relacionados solamente al  último viaje migratorio

atts_last_trip = [
  #Información personal
  'sex','age','statebrn','placebrn','marstat','edyrs',
  'occ','usyrl','usdurl','usdocl','usstatel','usplacel',
  'usmarl',
  #Características del último viaje a EE.UU. y experiencia migratoria
  'usoccl','uswagel','usbyl','ustrips',
  #Características sociales
  'reltrip','paistrip','sport','social','chicanos','blacks',
  'asians','anglos','latinos','lodging','english','enghome',
  'engwork','engfrnd','engneig',
  #Características financieras
  'finhelp','bankacct', 
  'card','taxes','consular','rent','food','remit',
  'remit1','remit2','remit3','remit4','remit5','savings', 'savretrn',
  'savings1','savings2','savings3','savings4',
  'savings5',
  #Servicios públicos y asistencia del gobierno
  'schools','unemp','foodst','welfare','afdc','wic','ssi',
  'general','doctor','hospital', 'hlthpmt1','hlthpmt2',
  'hlthpmt3','hlthpmt4',
  #Atributo de clase: att 'uscurtrp'
  'uscurtrp']

HRWAGE y HRWAGEO se podrían combinar?

In [5]:
mig_data_file = os.path.join(path_to_data, 'mig174.csv')
migother_file = os.path.join(path_to_data, 'migother174.csv')

#Lectura de los archivos
mig = pd.read_csv(mig_data_file, na_values=['9999',' '],dtype=dtypes_mig)
migother = pd.read_csv(migother_file, na_values=['9999',' '],dtype=dtypes_migother)

#Intersección de ambas tablas
intersec = pd.Index.intersection(mig.columns, migother.columns)
mig_migother = pd.concat([mig[intersec], migother[intersec]])
mig_migother.reset_index(drop=True, inplace=True)

In [6]:
migother_numeric_atts = ['weight', 'age', 'uswage1','uswagel','ustrips','crscst1', 'crscst2', 'crscst3', 'crscst4', 'crscst5', 
                'crscst6', 'crscst7', 'crscst8', 'crscst9', 'crscst10', 'crscst11', 'crscst12', 'crscst13', 'crscst14', 'crscst15', 'crscst16', 'crscst17', 'crscst18', 'crscst19', 
                'crscst20', 'crscst21', 'crscst22', 'crscst23', 'crscst24', 'crscst25', 'crscst26', 'crscst27', 'crscst28', 'crscst29', 'crscst30',
                'hrwageo', 'rent','food', 'remit', 'savings', 'savretrn', 'appcost','visacost', 'tripcost', 'retcost', 'houcost']

migother_cat_atts = [i for i in migother.columns if i not in migother_numeric_atts]

#dtypes_migother = {key: 'category' for key in migother_cat_atts}
#dtypes_migother.update({key: 'float64' for key in migother_numeric_atts})

In [7]:
mig_numeric_atts = ['weight', 'age', 'hhincome','ldowage','height','kgs','usdur1','usdurl','uswage1','uswagel','ustrips','usexp', 'crscst1', 'crscst2', 'crscst3', 'crscst4', 'crscst5', 
                'crscst6', 'crscst7', 'crscst8', 'crscst9', 'crscst10', 'crscst11', 'crscst12', 'crscst13', 'crscst14', 'crscst15', 'crscst16', 'crscst17', 'crscst18', 'crscst19', 
                'crscst20', 'crscst21', 'crscst22', 'crscst23', 'crscst24', 'crscst25', 'crscst26', 'crscst27', 'crscst28', 'crscst29', 'crscst30',
                'hrwage', 'hrwages', 'rent','food', 'remit', 'savings', 'savretrn', 'appcost','visacost', 'retcost', 'houcost']

mig_cat_atts = [i for i in mig.columns if i not in mig_numeric_atts]

#dtypes_mig = {key: 'category' for key in mig_cat_atts}
#dtypes_mig.update({key: 'float64' for key in mig_numeric_atts})

In [8]:
#Selección de los atributos correspondientes solo al último viaje migratorio
mig_migother = mig_migother[atts_last_trip]

#Selección de las instancias que no tienen valores faltantes en el atributo 'usyrl'
mig_migother = mig_migother.dropna(subset=['usyrl'])

#Selección de los personas que trabajaron por un salario durante el último viaje migratorio
mig_migother = mig_migother[mig_migother['uswagel'] != 8888]

#Ajuste de los salarios y gastos en USD de acuerdo al índice de inflación del último año que se realizó la encuesta
#utilizando la funcíón inflate de la librería cpi
adjustment_year = 2019
mig_migother['usyrl'] = mig_migother['usyrl'].astype(int)
results = []

print(f'Ajustando salarios y gastos en USD de acuerdo al índice de inflación del último año que se realizó la encuesta. ({adjustment_year})')
for att in ['uswagel', 'rent', 'food', 'remit', 'savings', 'savretrn']:
    tick = time.time()
    print(f'Ajustando {att}...',end=' ')
    #Utilizando multiprocessing para paralelizar el proceso de ajuste
    with Pool(processes=cpu_count()) as pool:
        #pool.starmap recibe una función y una lista de iterables con los argumentos de la función
        #pool.starmap(funcion, zip(arg1, arg2, arg3, ...)) -> funcion(arg1[0], arg2[0], arg3[0]), funcion(arg1[1], arg2[1], arg3[1]), ...
        r = pool.starmap(cpi.inflate, zip(mig_migother[att], mig_migother['usyrl'], [adjustment_year]*len(mig_migother)))
        results.append(pd.DataFrame(r, columns=[f'{att}_adj_{adjustment_year}']))
    tock = time.time()
    print(f'completado en {(tock-tick)/60:0.2f} mins.')
mig_migother = pd.concat([mig_migother, *results], axis=1)
mig_migother['usyrl'] = mig_migother['usyrl'].astype('category')
mig_migother.drop(['uswagel', 'rent', 'food', 'remit', 'savings', 'savretrn'], axis=1, inplace=True)

Ajustando salarios y gastos en USD de acuerdo al índice de inflación del último año que se realizó la encuesta. (2019)
Ajustando uswagel... completado en 1.62 mins.
Ajustando rent... completado en 1.60 mins.
Ajustando food... completado en 1.61 mins.
Ajustando remit... completado en 1.60 mins.
Ajustando savings... completado en 1.60 mins.
Ajustando savretrn... completado en 1.60 mins.


In [9]:
#Selección de las instancias que no tienen valores faltantes en el atributo 'uscurtrp'
mig_migother = mig_migother.dropna(subset=['uscurtrp'])

#Asignación del atributo 'uscurtrp' como etiqueta de clase
mig_migother['target'] = mig_migother['uscurtrp'].astype('category')
mig_migother.drop('uscurtrp', axis=1, inplace=True)
mig_migother['target'] = mig_migother.pop('target')


In [10]:
#Se eliminan los atributos que no corresponden a todas las comunidades encuestadas
mig_migother.drop(['card','taxes','consular','usmarl', 'asians','enghome', 'engwork', 'engfrnd', 
                   'engneig','afdc', 'wic', 'ssi', 'general','hlthpmt1', 'hlthpmt2', 'hlthpmt3', 'hlthpmt4'], axis=1, inplace=True)


In [11]:
mig_migother.to_csv(os.path.join(path_to_store, 'mig_migother.csv'), index=False)

In [12]:
cat_atts = ['sex', 'statebrn', 'placebrn', 'marstat', 'edyrs', 'occ', 'usyrl', 'usdurl', 'usdocl', 'usstatel', 'usplacel', 'usoccl', 'usbyl', 'reltrip', 'paistrip', 'sport', 'social', 'chicanos', 
            'blacks', 'anglos', 'latinos', 'lodging', 'english', 'finhelp', 'bankacct', 'remit1', 'remit2', 'remit3', 'remit4', 'remit5', 'savings1', 'savings2', 
            'savings3', 'savings4', 'savings5', 'schools', 'unemp', 'foodst', 'welfare', 'doctor', 'hospital']
j = [a for a in mig_migother.columns if a in mig_cat_atts and a in migother_cat_atts]
j


['sex',
 'statebrn',
 'placebrn',
 'marstat',
 'edyrs',
 'occ',
 'usyrl',
 'usdocl',
 'usstatel',
 'usplacel',
 'usoccl',
 'usbyl',
 'reltrip',
 'paistrip',
 'sport',
 'social',
 'chicanos',
 'blacks',
 'anglos',
 'latinos',
 'lodging',
 'english',
 'finhelp',
 'bankacct',
 'remit1',
 'remit2',
 'remit3',
 'remit4',
 'remit5',
 'savings1',
 'savings2',
 'savings3',
 'savings4',
 'savings5',
 'schools',
 'unemp',
 'foodst',
 'welfare',
 'doctor',
 'hospital']

In [13]:
[a for a in mig_migother.columns if a not in mig_cat_atts and a not in migother_cat_atts]

['age',
 'ustrips',
 'uswagel_adj_2019',
 'rent_adj_2019',
 'food_adj_2019',
 'remit_adj_2019',
 'savings_adj_2019',
 'savretrn_adj_2019',
 'target']